<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../_static/logo_stacked.png" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 3. Interactivity</h2></div>
<br><br>

In [ ]:
import pandas as pd
import param
import panel as pn

pn.extension('tabulator')

## **Interactivity**: Effectively using APIs for adding interactivity and reactivity

In the previous section we learned a lot about Param, which underlies not only the core architecture of Panel itself but also provides the foundation to adding interactivity to your applications. In this section we will discover how to leverage Parameters and dependencies on parameters to add interactivity. In particular we will focus on implementing interactivity through reactivity, rather than the more imperative style of programming you might be used to from other UI frameworks.

By the end of this tutorial you should have learned:

- How to use both declarative and imperative APIs to achieve interactivity
- How to develop both functional and class based interactive apps

## Declarative vs. Imperative programming

To build an interactive component in Panel we have two options, either we define callbacks that perform explicit actions, e.g. "when this widget changes update this component", or we declare a reactive function or method that returns a specific output given some input.

Let's look at what this looks like in practice by building a simple app that allows us to select a subset of columns to display in a table.

### Imperative

In both cases we start by loading our data and then defining a widget that will allow us to interact with the data:

In [ ]:
turbines = pd.read_parquet('./windturbines.parq')

cols = pn.widgets.MultiChoice(
    options=turbines.columns.to_list(), value=['p_name', 't_state', 't_county', 'p_year', 't_manu', 'p_cap'],
    width=500, height=120, name='Columns'
)

In the imperative approach we use `.param.watch` to set up a callback that will update the data when the widget changes:

In [ ]:
table = pn.widgets.Tabulator(turbines[cols.value], page_size=5)

def update_data(event):
    table.object = turbines[event.new]

cols.param.watch(update_data, 'value')

pn.Column(cols, table)

## Declarative

The declarative approach differs in that we only have to declare what we want to display and Panel takes care of the actual mechanics of updating the table for us:

In [ ]:
pn.Column(cols, pn.widgets.Tabulator(pn.rx(turbines)[cols], page_size=5))

### Exercise

Extend the app below by adding widgets that will let you to filter the data by year (`p_year`) and capacity (`p_cap`):

<details>
    <summary>Hint</summary>
    <code>pn.bind</code> accepts any number of arguments.
</details>

In [ ]:
cols = pn.widgets.MultiChoice(
    options=turbines.columns.to_list(), value=['p_name', 't_state', 't_county', 'p_year', 't_manu', 'p_cap'],
    width=500, height=120, name='Columns'
)

rxdf = pn.rx(turbines)

pn.Column(
    pn.Row(cols),
    pn.widgets.Tabulator(rxdf[cols], page_size=5, width=800)
)

Now briefly consider how this would look if you wrote this application using the imperative style of programming.

## Function vs. class based

The `pn.bind` API is an excellent way to quickly bind parameter values to functions and perform operations. However as soon as we want to track state or have many consumers of the output of such a dynamic function it can be hard to keep track. This is where classes come in. 

If you remember the [Param section](./01_Param.ipynb) a `Parameterized` class allows you to encapsulate state as parameters and those parameters can then be passed around to set up interactivity. Let's revive our `DataExplorer` class from that earlier lesson and see how we can structure a filtering application like above:

In [ ]:
import param

from panel.viewable import Viewer

class DataExplorer(Viewer):
    
    data = param.DataFrame(doc="Stores a DataFrame to explore")

    columns = param.ListSelector(default=['p_name', 't_state', 't_county', 'p_year', 't_manu', 'p_cap'])

    year = param.Range(default=(1981, 2022), bounds=(1981, 2022))
    
    capacity = param.Range(default=(0, 1100), bounds=(0, 1100))
    
    def __init__(self, **params):
        super().__init__(**params)
        self.param.columns.objects = self.data.columns.to_list()
    
    def __panel__(self):
        rxdf = self.param.data.rx()
        year = self.param.year.rx()
        cap = self.param.capacity.rx()
        filtered = rxdf[
            rxdf.p_year.between(*year) &
            rxdf.p_cap.between(*cap)
        ][self.param.columns]
        return pn.Column(
            pn.Row(
                pn.widgets.MultiChoice.from_param(self.param.columns, width=400),
                pn.Column(
                    self.param.year,
                    self.param.capacity
                )
            ),
            pn.widgets.Tabulator(
                filtered, page_size=10, pagination='remote'
            )
        )
    
DataExplorer(data=turbines)

As you can see `param.depends` allows us to set up a method that depends on specific parameters on the class (much like `pn.bind`) and then use that method as a proxy for the filtered data. An alternative approach would be to create a parameter to store the filtered data.

In [ ]:
class DataExplorer(Viewer):
    
    data = param.DataFrame(doc="Stores a DataFrame to explore")

    columns = param.ListSelector(default=['p_name', 't_state', 't_county', 'p_year', 't_manu', 'p_cap'])

    year = param.Range(default=(1981, 2022), bounds=(1981, 2022))
    
    capacity = param.Range(default=(0, 1100), bounds=(0, 1100))
    
    filtered = param.DataFrame(doc="Stores the filtered DataFrame")
    
    @param.depends('data', 'year', 'capacity', 'columns', watch=True, on_init=True)
    def filtered_data(self):
        self.filtered = self.data[self.data.p_year.between(*self.year) & self.data.p_cap.between(*self.capacity)][self.columns]

DataExplorer(data=turbines, year=(2021, 2022), capacity=(1000, 1100)).filtered.head()

This approach is beneficial if you have multiple consumers of this data, e.g. you generate a plot and a table both from this data.

### Exercise

Write an app that allows filtering the DataFrame and display both a table and a plot and caches the data on an intermediate parameter. You can use any plotting library you want.

<details>
    <summary>Hint</summary>
    <code>
    import hvplot.pandas
    
    df.iloc[:10000].hvplot.points('easting', 'northing', tiles='ESRI')
    </code>
</details>